In [1]:
# Do the needed imports
import skbio

# Declare some variables
table_fp = './final.only-16s.biom'
tree_name = './RR10856_placement.tog.tre'

In [2]:
inserted = skbio.TreeNode.read(tree_name)
# Make sure to change this path to match your system
# It needs to point to Greengenes 99% OTUs tree
gg_original = skbio.TreeNode.read('/Users/jose/qiime_software/gg_13_8_otus/trees/99_otus.tree')

In [3]:
# Add the taxonomy labels to the tree with the deblur sequences inserted
import functools
import operator

def cache_tip_names(tree):
    for node in tree.postorder():
        if node.is_tip():
            if node.name[0] in {'A', 'T', 'G', 'C'}:
                node.tip_names = frozenset([])
            else:
                node.tip_names = frozenset([node.name])
        else:
            node.tip_names = functools.reduce(operator.or_, [c.tip_names for c in node.children])
            
cache_tip_names(inserted)
cache_tip_names(gg_original)

In [4]:
clade_map = {n.tip_names: n.name for n in gg_original.non_tips() if n.name is not None and '__' in n.name}

In [5]:
mapped = 0

for n in inserted.preorder():
    if n.is_tip():
        continue
        
    if n.name is not None:
        print("unexpected")
        raise ValueError("unexpected name in sepp tree")
    
    if n.tip_names in clade_map:
        mapped += 1
        name = clade_map.pop(n.tip_names) 
        n.name = name
mapped

3124

In [6]:
inserted.write(tree_name + '.taxondecorated')

'./RR10856_placement.tog.tre.taxondecorated'

In [7]:
# Remove sequences from the BIOM table that did not insert
import biom
import h5py

table = biom.load_table(table_fp)
tree = skbio.TreeNode.read('./RR10856_placement.tog.tre.taxondecorated')

In [8]:
# construct a set of the tip names in the tree (inserted sequences are tips)
tree_tips = {n.name for n in tree.tips()}

# define a basic filtering function for the biom table
def filter_f(values, id_, md):
    return id_ in tree_tips

# retain only those observations which are in our tree
filtered_table = table.filter(filter_f, axis='observation', inplace=False)

# change the path to whatever works for you
with h5py.File('./final.only-16s.only-tree.biom', 'w') as fp:
    filtered_table.to_hdf5(fp, 'manual')

In [9]:
#Now check to see how many sequences did not insert:
ids = table.ids('observation')
set(ids) - tree_tips

set()

In [10]:
#After filtering out sequences that did not insert into the tree, now need to add taxonomy info to the biom table
#The code for this was written by Daniel McDonald
# to pull taxonomy

lineages = []

# for each tip in the tree
for node in tree.tips():
    lineage = []
    
    # for every ancestor of the tip
    for anc in node.ancestors():
        # make sure the ancestor has a name
        if anc.name is None:
            continue
            
        # this is dirty but basically the vertex names are not assured to 
        # just be taxon information. they sometimes also contain bootstrap
        # information.
        if anc.name[0] in ('0', '1'):  # does it appear to be a number
            if ':' in anc.name:
                bs, name = anc.name.split(':', 1)
            else:
                continue
        else:
            name = anc.name
        if '; ' in name:
            lineage.extend(name.split('; '))
        else:
            lineage.append(name)
        
    # store and reverse (as we were storing from species -> domain)
    lineages.append((node.name, lineage[::-1]))

In [11]:
# not all of the lineages are 7-level, so lets pad as necessary
default = ["%s__" % r for r in 'kpcofgs']
order = {d: i for i, d in enumerate(default)}

padded_lineages = []
for id_, lineage in lineages:
    padded = default[:]
    for taxon in lineage:
        rank = taxon[:3]
        padded[order[rank]] = taxon
    padded_lineages.append((id_, padded))

In [12]:
# write out the lineage information
with open('resulting_lineages.txt', 'w') as fp:
    fp.write("\n".join(['%s\t%s' % (id_, '; '.join(lin)) for id_, lin in padded_lineages]))
    fp.write('\n')
    
# add the lineage information to the filtered table
padded_lineages = {k: {'taxonomy': v} for k, v in padded_lineages if k in set(filtered_table.ids(axis='observation'))}
filtered_table.add_metadata(padded_lineages, axis='observation')

with h5py.File('final.only-16s.only-tree.with-tax.biom', 'w') as fp:
    filtered_table.to_hdf5(fp, 'manual')

In [13]:
!biom summarize-table -i ./final.only-16s.only-tree.with-tax.biom

Num samples: 463
Num observations: 1555
Total count: 8484381
Table density (fraction of non-zero values): 0.116

Counts/sample summary:
 Min: 2.0
 Max: 33635.0
 Median: 20097.000
 Mean: 18324.797
 Std. dev.: 6968.364
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
10856.BLANK5.12H: 2.0
10856.BLANK2.1H: 6.0
10856.BLANK3.2D: 7.0
10856.E20L.feces.45w.HFD.IgAkMup.3.5: 9.0
10856.E18L.feces.25w.NC.IgAkMup.1.24: 10.0
10856.BLANK5.12D: 11.0
10856.BLANK5.12F: 11.0
10856.BLANK5.12A: 11.0
10856.E24L.feces.48w.HFD.IgAkMup.10.18: 11.0
10856.BLANK5.12C: 11.0
10856.E49L.feces.26w.STAM.B6.PIGRK.3.3: 13.0
10856.E20L.feces.45w.HFD.IgAkCLM.3.9: 13.0
10856.E38L.feces.50w.NC.MupHe.4.17: 14.0
10856.E15L.feces.45w.HFD.IgAMup.5.18: 15.0
10856.E24L.feces.49w.HFD.IgAkCLM.6.9: 15.0
10856.BLANK5.12E: 16.0
10856.E24L.feces.49w.HFD.IgAkMup.3.8: 16.0
10856.E22L.feces.25w.HFD.MupHe.4.4: 16.0
10856.E20L.feces.45w.HFD.IgAkMup.4.8: 16.0
10856.E18L.feces.25w.NC